In [1]:
import pandas as pd
import emoji
import regex as re
import nltk
import string
from nltk.corpus import stopwords
from farasa.stemmer import FarasaStemmer
from tqdm import tqdm
from snowballstemmer import stemmer
tqdm.pandas()

nltk.download('stopwords')
df = pd.read_excel('C:/Users/Asus/Desktop/new pre/train2.xlsx')
df2 = pd.read_csv('C:/Users/Asus/Desktop/New folder (4)/test _no_label.csv')
print(df2.head())
df.head()

C:\Users\Asus\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


   ID                                 review_description
0   1  اهنئكم على خدمه العملاء في المحادثه المباشره م...
1   2  ممتاز جدا ولكن اتمنى ان تكون هناك بعض المسابقا...
2   3    كل محملته يقول تم ايقاف حطيت2 عشان تسوون الخطاء
3   4                                            شغل طيب
4   5                                         بعد ماجربت


,review_description,rating
0,شركه زباله و سواقين بتبرشم و مفيش حتي رقم للشك...,-1
1,خدمة الدفع عن طريق الكي نت توقفت عندي اصبح فقط...,1
2,تطبيق غبي و جاري حذفه ، عاملين اكواد خصم و لما...,-1
3,فعلا تطبيق ممتاز بس لو فى امكانية يتيح لمستخدم...,1
4,سيء جدا ، اسعار رسوم التوصيل لا تمت للواقع ب ص...,-1


In [2]:
def add_spaces_around_emojis(text):
    text=str(text)
    return re.sub(r'(\p{Emoji})', r' \1 ', text)



In [3]:
def remove_repeated_letters(text):
    result = ''
    count = 0
    for char in text:
        if result.endswith(char):
            count += 1
            if count < 2:
                result += char
        else:
            count = 1
            result += char
    return result


In [4]:
most_common_positive = 'متاز'
most_common_negative = 'فاشل'

positive_emojis = re.compile("[" + "".join([re.escape("\U0001F618"), re.escape("\U0001F61A"), re.escape("\U0001F63B"), re.escape("\U0001F648"),
                                            re.escape("\U0001F649"), re.escape("\U0001F64A"), re.escape("\U00002764\U0000FE0F"),
                                            re.escape("\U00002665"), re.escape("\U0001F44C"), re.escape("\U0001F44D"),
                                            re.escape("\U0001F44F"), re.escape("\U0001F493"), re.escape("\U0001F495"),
                                            re.escape("\U0001F496"), re.escape("\U0001F497"), re.escape("\U0001F498"),
                                            re.escape("\U0001F499"), re.escape("\U0001F49A"), re.escape("\U0001F49B"),
                                            re.escape("\U0001F49C"), re.escape("\U0001F49D"), re.escape("\U0001F49E"),
                                            re.escape("\U0001F49F"), re.escape("\U0001F5A4"), re.escape("\U00002763"), re.escape("\U0001F338"), re.escape("\U0001F60D")]) + "]")

negative_emojis = re.compile("[" + "".join([re.escape("\U0001F612"), re.escape("\U0001F613"), re.escape("\U0001F614"),
                                            re.escape("\U0001F615"), re.escape("\U0001F616"), re.escape("\U0001F620"),
                                            re.escape("\U0001F621"), re.escape("\U0001F622"), re.escape("\U0001F623"),
                                            re.escape("\U0001F624"), re.escape("\U0001F625"), re.escape("\U0001F44E"),
                                            re.escape("\U0001F4A9"), re.escape("\U0001F926"), re.escape("\U0001F494")]) + "]")




In [5]:
emojis = {
    "🙂":"يبتسم",
    "😂":"يضحك",
    "💔":"قلب حزين",
    "🙂":"يبتسم",
    "❤️":"حب",
    "❤":"حب",
    "😍":"حب",
    "😭":"يبكي",
    "😢":"حزن",
    "😔":"حزن",
    "♥":"حب",
    "💜":"حب",
    "😅":"يضحك",
    "🙁":"حزين",
    "💕":"حب",
    "💙":"حب",
    "😞":"حزين",
    "😊":"سعادة",
    "👏":"يصفق",
    "👌":"احسنت",
    "😴":"ينام",
    "😀":"يضحك",
    "😌":"حزين",
    "🌹":"وردة",
    "🙈":"حب",
    "😄":"يضحك",
    "😐":"محايد",
    "✌":"منتصر",
    "✨":"نجمه",
    "🤔":"تفكير",
    "😏":"يستهزء",
    "😒":"يستهزء",
    "🙄":"ملل",
    "😕":"عصبية",
    "😃":"يضحك",
    "🌸":"وردة",
    "😓":"حزن",
    "💞":"حب",
    "💗":"حب",
    "😑":"منزعج",
    "💭":"تفكير",
    "😎":"ثقة",
    "💛":"حب",
    "😩":"حزين",
    "💪":"عضلات",
    "👍":"موافق",
    "🙏🏻":"رجاء طلب",
    "😳":"مصدوم",
    "👏🏼":"تصفيق",
    "🎶":"موسيقي",
    "🌚":"صمت",
    "💚":"حب",
    "🙏":"رجاء طلب",
    "💘":"حب",
    "🍃":"سلام",
    "☺":"يضحك",
    "🐸":"ضفدع",
    "😶":"مصدوم",
    "✌️":"مرح",
    "✋🏻":"توقف",
    "😉":"غمزة",
    "🌷":"حب",
    "🙃":"مبتسم",
    "😫":"حزين",
    "😨":"مصدوم",
    "🎼 ":"موسيقي",
    "🍁":"مرح",
    "🍂":"مرح",
    "💟":"حب",
    "😪":"حزن",
    "😆":"يضحك",
    "😣":"استياء",
    "☺️":"حب",
    "😱":"كارثة",
    "😁":"يضحك",
    "😖":"استياء",
    "🏃🏼":"يجري",
    "😡":"غضب",
    "🚶":"يسير",
    "🤕":"مرض",
    "‼️":"تعجب",
    "🕊":"طائر",
    "👌🏻":"احسنت",
    "❣":"حب",
    "🙊":"مصدوم",
    "💃":"سعادة مرح",
    "💃🏼":"سعادة مرح",
    "😜":"مرح",
    "👊":"ضربة",
    "😟":"استياء",
    "💖":"حب",
    "😥":"حزن",
    "🎻":"موسيقي",
    "✒":"يكتب",
    "🚶🏻":"يسير",
    "💎":"الماظ",
    "😷":"وباء مرض",
    "☝":"واحد",
    "🚬":"تدخين",
    "💐" : "ورد",
    "🌞" : "شمس",
    "👆" : "الاول",
    "⚠️" :"تحذير",
    "🤗" : "احتواء",
    "✖️": "غلط",
    "📍"  : "مكان",
    "👸" : "ملكه",
    "👑" : "تاج",
    "✔️" : "صح",
    "💌": "قلب",
    "😲" : "مندهش",
    "💦": "ماء",
    "🚫" : "خطا",
    "👏🏻" : "برافو",
    "🏊" :"يسبح",
    "👍🏻": "تمام",
    "⭕️" :"دائره كبيره",
    "🎷" : "ساكسفون",
    "👋": "تلويح باليد",
    "✌🏼": "علامه النصر",
    "🌝":"مبتسم",
    "➿"  : "عقده مزدوجه",
    "💪🏼" : "قوي",
    "📩":  "تواصل معي",
    "☕️": "قهوه",
    "😧" : "قلق و صدمة",
    "🗨": "رسالة",
    "❗️" :"تعجب",
    "🙆🏻": "اشاره موافقه",
    "👯" :"اخوات",
    "©" :  "رمز",
    "👵🏽" :"سيده عجوزه",
    "🐣": "كتكوت",
    "🙌": "تشجيع",
    "🙇": "شخص ينحني",
    "👐🏽":"ايدي مفتوحه",
    "👌🏽": "بالظبط",
    "⁉️" : "استنكار",
    "⚽️": "كوره",
    "🕶" :"حب",
    "🎈" :"بالون",
    "🎀":    "ورده",
    "💵":  "فلوس",
    "😋":  "جائع",
    "😛":  "يغيظ",
    "😠":  "غاضب",
    "✍🏻":  "يكتب",
    "🌾":  "ارز",
    "👣":  "اثر قدمين",
    "❌":"رفض",
    "🍟":"طعام",
    "👬":"صداقة",
    "🐰":"ارنب",
    "☂":"مطر",
    "⚜":"مملكة فرنسا",
    "🐑":"خروف",
    "🗣":"صوت مرتفع",
    "👌🏼":"احسنت",
    "☘":"مرح",
    "😮":"صدمة",
    "😦":"قلق",
    "⭕":"الحق",
    "✏️":"قلم",
    "ℹ":"معلومات",
    "🙍🏻":"رفض",
    "⚪️":"نضارة نقاء",
    "🐤":"حزن",
    "💫":"مرح",
    "💝":"حب",
    "🍔":"طعام",
    "❤︎":"حب",
    "✈️":"سفر",
    "🏃🏻‍♀️":"يسير",
    "🍳":"ذكر",
    "🎤":"مايك غناء",
    "🎾":"كره",
    "🐔":"دجاجة",
    "🙋":"سؤال",
    "📮":"بحر",
    "💉":"دواء",
    "🙏🏼":"رجاء طلب",
    "💂🏿 ":"حارس",
    "🎬":"سينما",
    "♦️":"مرح",
    "💡":"قكرة",
    "‼":"تعجب",
    "👼":"طفل",
    "🔑":"مفتاح",
    "♥️":"حب",
    "🕋":"كعبة",
    "🐓":"دجاجة",
    "💩":"معترض",
    "👽":"فضائي",
    "☔️":"مطر",
    "🍷":"عصير",
    "🌟":"نجمة",
    "☁️":"سحب",
    "👃":"معترض",
    "🌺":"مرح",
    "🔪":"سكينة",
    "♨":"سخونية",
    "👊🏼":"ضرب",
    "✏":"قلم",
    "🚶🏾‍♀️":"يسير",
    "👊":"ضربة",
    "◾️":"وقف",
    "😚":"حب",
    "🔸":"مرح",
    "👎🏻":"لا يعجبني",
    "👊🏽":"ضربة",
    "😙":"حب",
    "🎥":"تصوير",
    "👉":"جذب انتباه",
    "👏🏽":"يصفق",
    "💪🏻":"عضلات",
    "🏴":"اسود",
    "🔥":"حريق",
    "😬":"عدم الراحة",
    "👊🏿":"يضرب",
    "🌿":"ورقه شجره",
    "✋🏼":"كف ايد",
    "👐":"ايدي مفتوحه",
    "☠️":"وجه مرعب",
    "🎉":"يهنئ",
    "🔕" :"صامت",
    "😿":"وجه حزين",
    "☹️":"وجه يائس",
    "😘" :"حب",
    "😰" :"خوف و حزن",
    "🌼":"ورده",
    "💋":  "بوسه",
    "👇":"لاسفل",
    "❣️":"حب",
    "🎧":"سماعات",
    "📝":"يكتب",
    "😇":"دايخ",
    "😈":"رعب",
    "🏃":"يجري",
    "✌🏻":"علامه النصر",
    "🔫":"يضرب",
    "❗️":"تعجب",
    "👎":"غير موافق",
    "🔐":"قفل",
    "👈":"لليمين",
    "™":"رمز",
    "🚶🏽":"يتمشي",
    "😯":"متفاجأ",
    "✊":"يد مغلقه",
    "😻":"اعجاب",
    "🙉" :"قرد",
    "👧":"طفله صغيره",
    "🔴":"دائره حمراء",
    "💪🏽":"قوه",
    "💤":"ينام",
    "👀":"ينظر",
    "✍🏻":"يكتب",
    "❄️":"تلج",
    "💀":"رعب",
    "😤":"وجه عابس",
    "🖋":"قلم",
    "🎩":"كاب",
    "☕️":"قهوه",
    "😹":"ضحك",
    "💓":"حب",
    "☄️ ":"نار",
    "👻":"رعب",
    "❎":"خطء",
    "🤮":"حزن",
    '🏻':"احمر"
    }

emoticons_to_emoji = {
    ":)" : "🙂",
    ":(" : "🙁",
    "xD" : "😆",
    ":=(": "😭",
    ":'(": "😢",
    ":'‑(": "😢",
    "XD" : "😂",
    ":D" : "🙂",
    "♬" : "موسيقي",
    "♡" : "❤",
    "☻"  : "🙂",
    }

In [6]:
def checkemojie(text):
    emojistext=[]
    for char in text:
        if any(emoji.distinct_emoji_list(char)) and char in emojis.keys():
            emojistext.append(emojis[emoji.distinct_emoji_list(char)[0]])
    return " ".join(emojistext)
def emojiTextTransform(text):
    cleantext=re.sub(r'[^\w\s]','',text)
    return cleantext+" "+checkemojie(text)

In [7]:
def remove_non_arabic_english(text):
    pattern = re.compile(r'[^\u0621-\u0652a-zA-Z\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF'
                         r'\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF'
                         r'\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\u2600-\u26FF\u2700-\u27BF\u2300-\u23FF ]+')
    return ' '.join(re.sub(pattern, "", text).split())

In [8]:
def normalize_emojis(text):
    text = re.sub(positive_emojis, most_common_positive, text)
    text = re.sub(negative_emojis, most_common_negative, text)
    return text

In [9]:

def remove_stopwords(text):
    # Get Arabic stopwords
    arabic_stopwords = set(stopwords.words('arabic'))
    
    # Remove 'لا' from the stopwords list
    arabic_stopwords.discard('لا')
    arabic_stopwords.discard('ليس')
    arabic_stopwords.discard('لم')
    arabic_stopwords.discard('لن')
    arabic_stopwords.discard('ما')
    arabic_stopwords.discard('ولاكن')
    arabic_stopwords.discard('ولكن')
    arabic_stopwords.discard('لاكن')
    arabic_stopwords.discard('لكن')
    arabic_stopwords.discard('مو')
    arabic_stopwords.discard('ليس')
    arabic_stopwords.discard('بس')
    arabic_stopwords.discard('لكن')
    arabic_stopwords.discard('غير')
    arabic_stopwords.discard('حتى')
    arabic_stopwords.discard('ولو')
    arabic_stopwords.discard('عدم')
    arabic_stopwords.add('جدا')
    # Tokenize the text into words
    words = text.split()
    
    # Remove stop words except 'لا'
    filtered_words = [word for word in words if word not in arabic_stopwords ]
    
    # Recreate the text without stop words
    filtered_text = ' '.join(filtered_words)
    
    return filtered_text


In [10]:
def remove_punctuation(text):
    # Define a translation table for removing punctuation
    translator = str.maketrans('', '', string.punctuation)
    
    # Remove punctuation using the translation table
    text_without_punct = text.translate(translator)
    
    return text_without_punct

In [11]:
def snowball_stemmer(text):

    arabic_stemmer = stemmer('arabic')
    tokens = text.split()
    stemmed_words = arabic_stemmer.stemWords(tokens)
    return ' '.join(stemmed_words)

In [12]:

df['review_description'] = df['review_description'].apply(add_spaces_around_emojis)
df['review_description'] = df['review_description'].apply(remove_repeated_letters)
df['review_description'] = df['review_description'].apply(emojiTextTransform)
df['review_description'] = df['review_description'].apply(remove_stopwords)
#df['review_description'] = df['review_description'].apply(remove_punctuation)

df['review_description'] = df['review_description'].progress_apply(snowball_stemmer)

100%|██████████████████████████████████████████████████████████████████████████| 39105/39105 [00:26<00:00, 1467.45it/s]


In [13]:

df2['review_description'] = df2['review_description'].apply(add_spaces_around_emojis)
df2['review_description'] = df2['review_description'].apply(remove_repeated_letters)
df2['review_description'] = df2['review_description'].apply(emojiTextTransform)
df2['review_description'] = df2['review_description'].apply(remove_stopwords)
#df2['review_description'] = df2['review_description'].apply(remove_punctuation)
df2['review_description'] = df2['review_description'].progress_apply(snowball_stemmer)

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1600.23it/s]


In [14]:
# Displaying the DataFrame
df.to_excel('C:/Users/Asus/Desktop/New folder (4)/train_test33.xlsx', index=False)
df2.to_csv('C:/Users/Asus/Desktop/New folder (4)/test_test33.csv', index=False)

In [15]:
df2.to_excel('C:/Users/Asus/Desktop/New folder (4)/test_test43.xlsx', index=False)